In [136]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

import timeit

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random


from dateutil.parser import parse

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

from sklearn import neighbors, datasets, tree, svm, preprocessing, utils
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss, classification_report, confusion_matrix, mean_squared_error, accuracy_score

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [397]:
def motorcycle_post_parser(total_listings):
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts'].find()[0:total_listings]
    
    title = []
    year = []
    price = []
    neighborhood = []
    description = []
    url = []
    #attributes
    model = []
    vin = []
    condition = []
    cryptocurrency = []
    delivery = []
    engine_size = []
    fuel_type = []
    mileage = []
    paint_color = []
    street_legal = []
    title_status = []
    transmission = []
    bike_type = []

    iterations = 0
    print('Starting Parsing')
    print(' ')
    for _ in post_html:         
        
        # attributes key words to parse through attributes
        vin_ = 'VIN:'
        condition_ = 'condition:'
        cryptocurrency_ = 'cryptocurrency'
        delivery_ = 'delivery'
        engine_size_ = 'engine displacement'
        fuel_type_ = 'fuel:'
        mileage_ = 'odometer:'
        paint_color_ = 'paint color:'
        street_legal_ = 'street legal'
        title_status_ = 'title status:'
        transmission_ = 'transmission:'
        bike_type_ = 'type:'  
        
        response = _['html']
        soup = BeautifulSoup(response, 'html.parser')

        try:
            post_title = soup.find('span', id = 'titletextonly')
            title.append(post_title.text)
        except:
            title.append(np.nan)
            
        try:
            post_price = soup.find('span', class_ = 'price').text
            price.append(float(post_price.lstrip('$')))
        except:
            price.append(np.nan)
        
        try:
            post_neighborhood = soup.find('small')
            neighborhood.append(((post_neighborhood.text).replace(' (', '')).replace(')', ''))
        except:
            neighborhood.append(np.nan)
        
        try:
            post_description = soup.find('section', id = 'postingbody')
            description.append((post_description.text).replace('\n\nQR Code Link to This Post\n\n\n', ''))
        except:
            description.append(np.nan)
        
        try:
            year_ = parse(post_title.text, fuzzy=True).year
            if 1920 < year_ < 2022:
                year.append(year_)
            else:
                year.append(np.nan)
        except:
            year.append(np.nan)
        
        url.append(_['_id'])
        
        post_attributes = str(soup.find_all('p', attrs = {'class': 'attrgroup'}))
        post_attributes = post_attributes.replace(post_attributes, str(post_attributes.replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''','').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '').replace('''<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]''', '')))
                    
        bike_attributes = (post_attributes.split(','))

        vin_res = [i for i in bike_attributes if vin_ in i]
        condition_res = [i for i in bike_attributes if condition_ in i]
        cryptocurrency_res = [i for i in bike_attributes if cryptocurrency_ in i]
        delivery_res = [i for i in bike_attributes if delivery_ in i]
        engine_size_res = [i for i in bike_attributes if engine_size_ in i]
        fuel_type_res = [i for i in bike_attributes if fuel_type_ in i]
        mileage_res = [i for i in bike_attributes if mileage_ in i]
        paint_color_res = [i for i in bike_attributes if paint_color_ in i]
        street_legal_res = [i for i in bike_attributes if street_legal_ in i]
        title_status_res = [i for i in bike_attributes if title_status_ in i]
        transmission_res = [i for i in bike_attributes if transmission_ in i]
        bike_type_res = [i for i in bike_attributes if bike_type_ in i]
        
        #     
        
        try:
            model.append(bike_attributes[0])
        except:
            model.append('')
                
        try:
            vin.append((vin_res[0]).replace((vin_res[0], '1')))
        except:
            vin.append(0)
            
        try:
            condition.append(str(condition_res[0]).replace('condition:', ''))
        except:
            condition.append('good')
               
        try:
            cryptocurrency.append((cryptocurrency_res[0]).replace((cryptocurrency_res[0]), '1'))
        except:
            cryptocurrency.append(0)
        
        try:
            delivery.append((delivery_res[0]).replace((delivery_res[0]), '1'))
        except:
            delivery.append(0)
                
        try:
            engine_size.append(float(str(engine_size_res[0]).replace('engine displacement (CC):', '')))
        except:
            engine_size.append(np.nan)
            
        try:
            fuel_type.append(str(fuel_type_res[0]).replace('fuel:', ''))
        except:
            fuel_type.append('gas') #gas is the most common
            
        try:
            mileage.append(float(str(mileage_res[0]).replace('odometer:', '')))
        except:
            mileage.append(np.nan)
                    
        try:
            paint_color.append(str(paint_color_res[0]).replace('paint color:', ''))
        except:
            paint_color.append('black') #black is the most common
                 
        try:
            street_legal.append((street_legal_res[0]).replace(street_legal_res[0], '1'))
        except:
            street_legal.append(0)
    
        try:
            title_status.append(str(title_status_res[0]).replace('title status:', ''))
        except:
            title_status.append('clean') #clean is the most common
                
        try: 
            transmission.append(str(transmission_res[0]).replace('transmission:', '')[0:5]) #we are pulling the first five letters to cut out the html
        except:
            transmission.append('manual') 

        try:
            bike_type.append(str(bike_type_res[0]).replace('type:', '')[0:5]) #we are pulling the first five letters to cut out the html
        except:
            bike_type.append('')
                
        iterations += 1

        if iterations%1000 == 0:
            print(str(round(100*round(iterations/total_listings, 2),2)) + '%' + ' parsed succesfully')
    print(' ')
    print('Parsing Complete, Preparing Dataframe')
    df_dictionary = { 'title': title, 'year': year, 'price': price, 'neighborhood': neighborhood, 'description': description, 'url':url, 'model':model, 'vin': vin, 'condition': condition, 'cryptocurrency': cryptocurrency, 'delivery': delivery, 'engine_size': engine_size, 'fuel_type': fuel_type, 'mileage': mileage, 'color': paint_color, 'street_legal': street_legal, 'title_status': title_status, 'transmission': transmission, 'bike_type': bike_type}
    df = pd.DataFrame(df_dictionary)
    
#   Cleaning DataFrame 
    df['neighborhood'].replace('\n       google map\n        ', '', inplace=True)
    
    print(' ')
    print('Parsing Complete! Enjoy Mammal!')
    return df

In [398]:
moto_df = motorcycle_post_parser(58600)

Starting Parsing
 
2.0% parsed succesfully
3.0% parsed succesfully
5.0% parsed succesfully
7.0% parsed succesfully
9.0% parsed succesfully
10.0% parsed succesfully
12.0% parsed succesfully
14.0% parsed succesfully
15.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname V identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


17.0% parsed succesfully
19.0% parsed succesfully
20.0% parsed succesfully
22.0% parsed succesfully
24.0% parsed succesfully
26.0% parsed succesfully
27.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname CB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


29.0% parsed succesfully


...

48.0% parsed succesfully
49.0% parsed succesfully
51.0% parsed succesfully
53.0% parsed succesfully
55.0% parsed succesfully
56.0% parsed succesfully
58.0% parsed succesfully
60.0% parsed succesfully
61.0% parsed succesfully
63.0% parsed succesfully
65.0% parsed succesfully
67.0% parsed succesfully
68.0% parsed succesfully
70.0% parsed succesfully
72.0% parsed succesfully
73.0% parsed succesfully
75.0% parsed succesfully
77.0% parsed succesfully
78.0% parsed succesfully
80.0% parsed succesfully
82.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname D identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


84.0% parsed succesfully
85.0% parsed succesfully
87.0% parsed succesfully
89.0% parsed succesfully
90.0% parsed succesfully
92.0% parsed succesfully


/home/eric/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname K identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


94.0% parsed succesfully
96.0% parsed succesfully
97.0% parsed succesfully
99.0% parsed succesfully
 
Parsing Complete, Preparing Dataframe
 
Parsing Complete! Enjoy Mammal!


In [468]:
df = moto_df.copy()

In [400]:
moto_df.to_csv('craigslist_motorcycle_data_full')

In [469]:
df.head()

,title,year,price,neighborhood,description,url,model,vin,condition,cryptocurrency,delivery,engine_size,fuel_type,mileage,color,street_legal,title_status,transmission,bike_type
0,2016 Honda CBR300r,2016.0,2900.0,Auburn,"2016 Honda CBR300r\n6,186 miles as of listing\...",https://auburn.craigslist.org/mcy/d/auburn-uni...,2016 honda cbr300r,0,excellent,0,0,286.0,gas,NaN,yellow,1,clean,manua,sport
1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",NaN,NaN,FAYETTEVILLE,"WE SHIP NATIONWIDE, FINANCE NATIONWIDE! YOU SE...",https://auburn.craigslist.org/mcd/d/fayettevil...,all bikes,0,excellent,0,0,NaN,gas,NaN,black,0,clean,manua,
2,2017 Yamaha FZ 07 very low miles,2017.0,5000.0,Near Montgomery,"2017 Yamaha FZ-07 with only 1100 miles, bike i...",https://auburn.craigslist.org/mcy/d/montgomery...,2017 Yamaha FZ-07,0,like new,0,0,700.0,gas,1100.0,silver,0,clean,manua,stree
3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,NaN,NaN,,2010 Yellow GL1800 CSC Viper\n\nLOOK!!! only 8...,https://auburn.craigslist.org/mcd/d/haleyville...,2010 Honda GL1800 CSC,0,like new,0,0,1832.0,gas,8209.0,yellow,0,clean,manua,trike
4,2006 Honda XR650L *Price Drop*,2006.0,3800.0,Auburn,"2006 Honda XR650L \n\n8,000 miles. This number...",https://auburn.craigslist.org/mcy/d/auburn-200...,2006 honda xr650l,0,excellent,0,0,650.0,gas,8100.0,red,1,clean,manua,dual-


In [470]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58600 entries, 0 to 58599
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           58595 non-null  object 
 1   year            28598 non-null  float64
 2   price           56652 non-null  float64
 3   neighborhood    58483 non-null  object 
 4   description     58595 non-null  object 
 5   url             58600 non-null  object 
 6   model           58600 non-null  object 
 7   vin             58600 non-null  int64  
 8   condition       58600 non-null  object 
 9   cryptocurrency  58600 non-null  object 
 10  delivery        58600 non-null  object 
 11  engine_size     23605 non-null  float64
 12  fuel_type       58600 non-null  object 
 13  mileage         43120 non-null  float64
 14  color           58600 non-null  object 
 15  street_legal    58600 non-null  object 
 16  title_status    58600 non-null  object 
 17  transmission    58600 non-null 

In [471]:
for idx, i in enumerate(df['price']):
    if i > 300000 or i < 100:
        df.drop(idx, axis=0, inplace = True)

In [472]:
df = df.reset_index(drop=True)

In [474]:
for idx, i in enumerate(df['engine_size']):
    if i > 2500:
        df.drop(idx, axis=0, inplace = True)

In [475]:
df = df.reset_index(drop=True)

In [476]:
df['engine_size'].replace(np.nan, df['engine_size'].mean(), inplace=True)

In [477]:
for idx, i in enumerate(df['mileage']):
    if i > 500000:
        df.drop(idx, axis=0, inplace = True)

In [478]:
df = df.reset_index(drop=True)

In [479]:
for idx, i in enumerate(df['condition'][:100]):
    df['condition'].replace({'new': '6'}, inplace=True)
    df['condition'].replace({'like new': '5'}, inplace=True)
    df['condition'].replace({'excellent': '4'}, inplace=True)
    df['condition'].replace({'good': '3'}, inplace=True)
    df['condition'].replace({'fair': '2'}, inplace=True)
    df['condition'].replace({'salvage': '1'}, inplace=True)

In [480]:
df['condition'] = pd.to_numeric(df['condition'])

In [481]:
df = df.reset_index(drop=True)

In [483]:
df['mileage'].replace(np.nan, df['mileage'].mean(), inplace=True)

In [484]:
df['delivery'] = pd.to_numeric(df['delivery'])

In [485]:
df['street_legal'] = pd.to_numeric(df['street_legal'])

In [486]:
df.dropna(inplace=True)

In [487]:
df = df.reset_index(drop=True)

In [489]:
# condition_dummies = pd.get_dummies(df['condition'])
fuel_dummies = pd.get_dummies(df['fuel_type'])
color_dummies = pd.get_dummies(df['color'])
# title_dummies = pd.get_dummies(df['title_status'])
# transmission_dummies = pd.get_dummies(df['transmission'])
bike_type_dummies = pd.get_dummies(df['bike_type'])
df = pd.concat([df, bike_type_dummies], axis = 1)
# df = pd.concat([df, transmission_dummies], axis = 1)
# df = pd.concat([df, title_dummies], axis = 1)
df = pd.concat([df, color_dummies], axis = 1)
df = pd.concat([df, fuel_dummies], axis = 1)

In [490]:
df.head()

,title,year,price,neighborhood,description,url,model,vin,condition,cryptocurrency,...,red,silver,white,yellow,,diesel,electric,gas,hybrid,other
0,2016 Honda CBR300r,2016.0,2900.0,Auburn,"2016 Honda CBR300r\n6,186 miles as of listing\...",https://auburn.craigslist.org/mcy/d/auburn-uni...,2016 honda cbr300r,0,4,0,...,0,0,0,1,0,0,0,1,0,0
1,2017 Yamaha FZ 07 very low miles,2017.0,5000.0,Near Montgomery,"2017 Yamaha FZ-07 with only 1100 miles, bike i...",https://auburn.craigslist.org/mcy/d/montgomery...,2017 Yamaha FZ-07,0,5,0,...,0,1,0,0,0,0,0,1,0,0
2,2006 Honda XR650L *Price Drop*,2006.0,3800.0,Auburn,"2006 Honda XR650L \n\n8,000 miles. This number...",https://auburn.craigslist.org/mcy/d/auburn-200...,2006 honda xr650l,0,4,0,...,1,0,0,0,0,0,0,1,0,0
3,2016 Harley Davidson Roadster Sportster for Sale,2016.0,6999.0,Auburn,2016 Harley-Davidson Sportster Roadster (EVO 1...,https://auburn.craigslist.org/mcy/d/auburn-201...,2016 harley-davidson roadster,0,4,0,...,1,0,0,0,0,0,0,1,0,0
4,2008 Harley Davidson Heritage Softail Low Miles,2008.0,5000.0,Tuscaloosa,2008 Harley Davidson Heritage Softail With Onl...,https://bham.craigslist.org/mcy/d/tuscaloosa-2...,2008 harley davidson heritage classic,0,3,0,...,1,0,0,0,0,0,0,1,0,0


In [498]:
df.drop(df.columns[19], axis=1, inplace=True)

In [500]:
df.to_csv('motorcycle_model_ready_data')

In [501]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27769 entries, 0 to 27768
Data columns (total 50 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           27769 non-null  object 
 1   year            27769 non-null  float64
 2   price           27769 non-null  float64
 3   neighborhood    27769 non-null  object 
 4   description     27769 non-null  object 
 5   url             27769 non-null  object 
 6   model           27769 non-null  object 
 7   vin             27769 non-null  int64  
 8   condition       27769 non-null  int64  
 9   cryptocurrency  27769 non-null  object 
 10  delivery        27769 non-null  int64  
 11  engine_size     27769 non-null  float64
 12  fuel_type       27769 non-null  object 
 13  mileage         27769 non-null  float64
 14  color           27769 non-null  object 
 15  street_legal    27769 non-null  int64  
 16  title_status    27769 non-null  object 
 17  transmission    27769 non-null 